In [ ]:
#query building functions and objects

class RareCodingVariants(object):
    
    def __init__(self, table, suffix = "_rare_coding"):
        self.inputTable = table
        self.tablename = table + suffix
        
    def buildTables(self):
        temptable = self.tablename + "_temp"
        query = ""
        query += "CREATE TABLE " + temptable + " as SELECT * from " + self.inputTable + " where \
\n(AfricanHapMapFreq < 1.0) and (AsianHapMapFreq < 1.0) and (EuropeanHapMapFreq < 1.0) and \
\n(dbSNP_5_percent_in_all != 'TRUE') and (dbSNP_5_percent_in_any != 'TRUE') and \
\n(F_rarest_allele < 0.01) and (MAFinESP < 0.01) and (functionGVS NOT IN \
\n('intron' , 'synonymous', '5-prime-UTR', '3-prime-UTR', 'non-coding-exon', \
\n'upstream-gene', 'downstream-gene', 'intergenic')) \
\nGROUP by varMD5; \
\nDELETE from " + self.inputTable + "VCF where instr(uniLocVCF, 'GL') != 0; \
\nCREATE TABLE " + self.tablename + " as select * from " + temptable + " left join " + self.inputTable + "VCF on varMD5 = varMD5VCF; \
\nDROP TABLE " + temptable + "; \
\nALTER TABLE " + self.tablename + " ADD INDEX (varMD5), ADD INDEX (uniLoc), ADD INDEX (geneList);"
        return query
    
    def getVariants(self):
        query = "SELECT * FROM " + self.tablename + ";"
        return query
    
    def countVariants(self, unique = False):
        query = ""
        if unique:
            query += "SELECT COUNT(distinct varMD5) "
        if not unique:
            query += "SELECT COUNT(*) "
        query += "from " + self.tablename + ";"
        return query

    
class KnownGenes(object):
    
    def __init__(self, table, knownGeneList):
        self.table = table
        try:
            assert hasattr(knownGeneList, '__iter__') and not type(knownGeneList) is str
        except:
            raise TypeError("List of known genes must be passed as a list or a tupple")
        self.knownGeneList = list(knownGeneList)
        self.knownGeneList = [gene.strip() for gene in self.knownGeneList]
        self.knownGeneList = ["'" + gene + "'," for gene in self.knownGeneList]
        
    def listFromFile(file):
        if not hasattr(file, 'read'):
            raise TypeError("Object passed must be a file")
        linecollector = []
        line = file.readline()
        while line:
            if not line[0] == '#':
                linecollector.append(line.strip())
            line = file.readline()
        return linecollector
    
    def hashedListFromFile(file):
        if not hasattr(file, 'read'):
            raise TypeError("Object passed must be a file")
        listcollector = {}
        line = file.readline()
        line = line.strip()
        if line[0] != "#":
            raise RuntimeError("First line must be a group name and start with a # symbol")
        while line:
            if line[0] == "#":
                currentlist = line.replace("#","")
                currentlist = currentlist.replace("\t"," ")
                listcollector[currentlist] = []
            else:
                listcollector[currentlist].append(line.strip())
            line = file.readline()
            line = line.strip()
        return listcollector
       
    def getVariants(self):
        knownGeneString = " ".join(self.knownGeneList)
        knownGeneString = knownGeneString.strip()
        knownGeneString = knownGeneString.strip(",")
        query = ""
        query += "SELECT * FROM " + self.table + " where geneList in (" + knownGeneString + ") ORDER by geneList;"
        return query
    

class Homozygous(object):
    
    def __init__(self, table, readCutoff = 0.1, suffix = "_homozygosity"):
        self.inputTable = table
        self.tablename = table + suffix
        if readCutoff:
            try:
                readCutoff = float(readCutoff)
            except TypeError:
                raise TypeError("Read cutoff value must be a number.")
            if readCutoff >= 1.0:
                readCutoff = readCutoff/100
            try:
                assert readCutoff <= 1.0
            except:
                raise ValueError("Percent for cutoff must be below 100")
            if readCutoff > 0.5:
                readCutoff = 1.0 - readCutoff
        self.readCutoff = readCutoff
        
    def buildTables(self):
        query = ""
        query += "CREATE TABLE " + self.tablename + " as SELECT * from " + self.inputTable + " \
\nWHERE sampleGenotype NOT IN ('S' , 'R', 'Y', 'M', 'K', 'W') \
\nAND (INSTR(sampleGenotype, '/') = 0 OR substring_index(sampleGenotype, '/', 1) = substring_index(sampleGenotype, '/', -1))"
        if self.readCutoff:
            query += "\nOR (Ref_read_percentVCF < " + str(round(self.readCutoff, 4)) + ");\n"
        else:
            query += ";\n"
        query += "ALTER TABLE " + self.tablename + " ADD INDEX (varMD5), ADD INDEX (uniLoc), ADD INDEX (geneList);"
        return query
   
    def getVariants(self):
        query = "SELECT * FROM " + self.tablename + ";"
        return query
       
    def countVariants(self, unique = False):
        query = ""
        if unique:
            query += "SELECT COUNT(distinct varMD5) "
        if not unique:
            query += "SELECT COUNT(*) "
        query += "from " + self.tablename + ";"
        
        
class CompoundHet(object):
    
    def __init__(self, table, suffix = "_compound_het"):
        self.inputTable = table
        self.tablename = table + suffix
        
    def buildTables(self):
        hitcounts = self.inputTable + "_hit_count"
        query = ""
        query += "CREATE TABLE " + hitcounts + " as \
\nSELECT distict varMD5, uniLoc, geneList, count(*) hits \
\nFROM " + self.inputTable + " group by geneList; \
\nCREATE TABLE " + self.tablename + " as SELECT * from " + self.inputTable + " \
\nWHERE geneList in (SELECT distinct geneList from " + hitcounts + " WHERE \
hits > 1) \
\nORDER by geneList asc;"
        return query
    
    def getVariants(self):
        query = "SELECT * FROM " + self.tablename + ";"
        return query
        
    def countVariants(self, unique = False):
        query = ""
        if unique:
            query += "SELECT COUNT(distinct varMD5) "
        if not unique:
            query += "SELECT COUNT(*) "
        query += "from " + self.tablename + ";"
        return query
    
    def countGenes(self):
        query = ""
        query += "SELECT COUNT(distinct geneList)from " + self.tablename + ";"
        return query
    
    
class DeNovoChanges(object):
    
    def __init__(self, proband, parent1 = "", parent2 = "", group = ""):
        if not (proband and parent1 and parent2) or not type(proband) is str:
            try:
                assert hasattr(proband, '__iter__') and len(proband) == 3
                self.group = parent1
                self.parent2 = proband[2]
                self.parent1 = proband[1]
                self.proband = proband[0]
            except:
                raise RuntimeError("Finding changed variants requires passing proband and original as either arguments or a tupple of two identifiers")
        else:
            self.proband = proband
            self.group = group
            self.parent1 = parent1
            self.parent2 = parent2            
        
    def buildTables(self):
        query = ""        
        query += "DELETE FROM " + self.proband + "VCF where instr(uniLocVCF, 'GL') != 0;\n"
        query += "DELETE FROM " + self.parent1 + "VCF where instr(uniLocVCF, 'GL') != 0;\n"
        
        query += "CREATE TABLE " + self.proband + self.group + "_genotemp1 as select uniLoc, varMD5, geneList, GenotypeVCF proband_geno, \
\nfunctionGVS, functionDBSNP, aminoAcids, proteinPosition, distanceToSplice, polyPhen, granthamScore, scorePhastCons, scoreCADD, \
\nF_rarest_allele, MAFinESP, FILTERVCF, \
\nGenotype_QualityVCF, Variant_Confidence_Quality_by_DepthVCF, \
\nDepth_CountedVCF depth_proband, Reference_readsVCF Ref_reads_proband, Alt_readsVCF Alt_read_proband, \
\nRef_read_percentVCF Ref_read_pct_proband, A_readVCF A_read_proband, G_readVCF G_read_proband, \
\nT_readVCF T_read_proband, C_readVCF C_read_proband, A_percentVCF A_percent_proband, G_percentVCF G_percent_proband, \
\nT_percentVCF T_percent_proband, C_percentVCF C_percent_proband from " + self.proband + self.group + "; \n"  
        query += "ALTER TABLE " + self.proband + self.group + "_genotemp1 ADD INDEX (uniLoc), ADD INDEX (geneList), ADD INDEX(varMD5);\n"
        query += "CREATE TABLE " + self.proband + self.group + "_genotemp2 as select " + self.proband + self.group + "_genotemp1.*, \
\n" + self.parent1 + "VCF.GenotypeVCF p1_geno, " + self.parent1 + "VCF.Depth_CountedVCF Depth_p1, \
\n" + self.parent1 + "VCF.Reference_readsVCF Ref_read_p1, " + self.parent1 + "VCF.Alt_readsVCF Alt_read_p1, \
\n" + self.parent1 + "VCF.Ref_read_percentVCF Ref_read_pct_p1, " + self.parent1 + "VCF.A_readVCF A_read_p1, \
\n" + self.parent1 + "VCF.G_readVCF G_read_p1, " + self.parent1 + "VCF.T_readVCF T_read_p1, " + self.parent1 + "VCF.C_readVCF C_read_p1,\
\n" + self.parent1 + "VCF.A_percentVCF A_percent_p1, " + self.parent1 + "VCF.G_percentVCF G_percent_p1, " + self.parent1 + "VCF.T_percentVCF T_percent_p1, \
\n" + self.parent1 + "VCF.C_percentVCF C_percent_p1 from " + self.proband + self.group + "_genotemp1 left join " + self.parent1 + "VCF on \
\n" + self.proband + self.group + "_genotemp1.varMD5 = " + self.parent1 + "VCF.varMD5VCF;\n"
        query += "DROP TABLE " + self.proband + self.group + "_genotemp1;\n"
        query += "ALTER TABLE " + self.proband + group  + "_genotemp2 ADD INDEX (uniLoc), ADD INDEX (geneList), ADD INDEX(varMD5);\n"
        query += "CREATE TABLE " + self.proband + self.group + "_genotypes as select " + self.proband + self.group + "_genotemp2.*, \
\n" + self.parent2 + "VCF.GenotypeVCF p2_geno, " + self.parent2 + "VCF.Depth_CountedVCF Depth_p2, \
\n" + self.parent2 + "VCF.Reference_readsVCF Ref_read_p2, " + self.parent2 + "VCF.Alt_readsVCF Alt_read_p2, \
\n" + self.parent2 + "VCF.Ref_read_percentVCF Ref_read_pct_p2, " + self.parent2 + "VCF.A_readVCF A_read_p2, \
\n" + self.parent2 + "VCF.G_readVCF G_read_p2, " + self.parent2 + "VCF.T_readVCF T_read_p2, " + self.parent2 + "VCF.C_readVCF C_read_p2,\
\n" + self.parent2 + "VCF.A_percentVCF A_percent_p2, " + self.parent2 + "VCF.G_percentVCF G_percent_p2, " + self.parent2 + "VCF.T_percentVCF T_percent_p2, \
\n" + self.parent2 + "VCF.C_percentVCF C_percent_p2 from " + self.proband + self.group + "_genotemp2 left join " + self.parent2 + "VCF on \
\n" + self.proband + self.group + "_genotemp2.varMD5 = " + self.parent2 + "VCF.varMD5VCF;\n"
        query += "DROP TABLE " + self.proband + self.group + "_genotemp2;\n"
        query += "ALTER TABLE " + self.proband + group  + "_genotypes ADD INDEX (uniLoc), ADD INDEX (geneList), ADD INDEX(varMD5);\n"
        self.genotable = self.proband + self.group + "_genotypes"
        return query
    
    def getVariants(self, stringent = False):
        query = ""
        query += "SELECT * FROM " + self.proband + self.group + "_genotypes WHERE \
\n(substring_index(proband_geno, '/', 1) != substring_index(proband_geno, '/', -1)) and \
\n((substring_index(proband_geno, '/', 1) != '0' and substring_index(proband_geno, '/', 1) not in \
\n (substring_index(mom_geno, '/', 1), substring_index(mom_geno, '/', -1), \
\n  substring_index(dad_geno, '/', 1), substring_index(dad_geno, '/', -1))) \
\nor \
\n(substring_index(proband_geno, '/', -1) != '0' and substring_index(proband_geno, '/', -1) not in \
\n (substring_index(mom_geno, '/', 1), substring_index(mom_geno, '/', -1), \
\n  substring_index(dad_geno, '/', 1), substring_index(dad_geno, '/', -1)) \
\n))"
        if not stringent:
            query += ";\n"
        if stringent:
            query += "\nand proband_geno != './.' and p1_geno != './.' and p2_geno != './.';\n"
        return query   
    
    def countVariants(self, stringent = False, unique = False):
        query = ""
        if unique:
            query += "SELECT COUNT(distinct varMD5) FROM "
        if not unique:
            query += "SELECT COUNT(*) FROM "
        query += self.proband + self.group + "_genotypes WHERE \
\n(substring_index(proband_geno, '/', 1) != substring_index(proband_geno, '/', -1)) and \
\n((substring_index(proband_geno, '/', 1) != '0' and substring_index(proband_geno, '/', 1) not in \
\n (substring_index(mom_geno, '/', 1), substring_index(mom_geno, '/', -1), \
\n  substring_index(dad_geno, '/', 1), substring_index(dad_geno, '/', -1))) \
\nor \
\n(substring_index(proband_geno, '/', -1) != '0' and substring_index(proband_geno, '/', -1) not in \
\n (substring_index(mom_geno, '/', 1), substring_index(mom_geno, '/', -1), \
\n  substring_index(dad_geno, '/', 1), substring_index(dad_geno, '/', -1)) \
\n))"
        if not stringent:
            query += ";\n"
        if stringent:
            query += "\nand proband_geno != './.' and p1_geno != './.' and p2_geno != './.';\n"
        return query 
    
    
class SomaticChanges(object):
    
    def __init__(self, proband, parent1 = "", group = ""):
        if not (proband and parent1) or not type(proband) is str:
            try:
                assert hasattr(proband, '__iter__') and len(proband) == 2
                self.group = parent1
                self.parent1 = proband[1]
                self.proband = proband[0]
            except:
                raise RuntimeError("Finding changed variants requires passing proband and original as either arguments or a tupple of two identifiers")
        else:
            self.proband = proband
            self.group = group
            self.parent1 = parent1
        
    def buildTables(self):
        query = ""        
        query += "DELETE FROM " + self.proband + "VCF where instr(uniLocVCF, 'GL') != 0;\n"
        query += "DELETE FROM " + self.parent1 + "VCF where instr(uniLocVCF, 'GL') != 0;\n"
        query += "CREATE TABLE " + self.proband + self.group + "_genotemp1 as select uniLoc, varMD5, geneList, GenotypeVCF proband_geno, \
\nfunctionGVS, functionDBSNP, aminoAcids, proteinPosition, distanceToSplice, polyPhen, granthamScore, scorePhastCons, scoreCADD, \
\nF_rarest_allele, MAFinESP, FILTERVCF, \
\nGenotype_QualityVCF, Variant_Confidence_Quality_by_DepthVCF, \
\nDepth_CountedVCF depth_proband, Reference_readsVCF Ref_reads_proband, Alt_readsVCF Alt_read_proband, \
\nRef_read_percentVCF Ref_read_pct_proband, A_readVCF A_read_proband, G_readVCF G_read_proband, \
\nT_readVCF T_read_proband, C_readVCF C_read_proband, A_percentVCF A_percent_proband, G_percentVCF G_percent_proband, \
\nT_percentVCF T_percent_proband, C_percentVCF C_percent_proband from " + self.proband + self.group + "; \n"  
        query += "ALTER TABLE " + self.proband + self.group + "_genotemp1 ADD INDEX (uniLoc), ADD INDEX (geneList), ADD INDEX(varMD5);\n"
        query += "CREATE TABLE " + self.proband + self.group + "_genotypes as select " + self.proband + self.group + "_genotemp1.*, \
\n" + self.parent1 + "VCF.GenotypeVCF p1_geno, " + self.parent1 + "VCF.Depth_CountedVCF Depth_p1, \
\n" + self.parent1 + "VCF.Reference_readsVCF Ref_read_p1, " + self.parent1 + "VCF.Alt_readsVCF Alt_read_p1, \
\n" + self.parent1 + "VCF.Ref_read_percentVCF Ref_read_pct_p1, " + self.parent1 + "VCF.A_readVCF A_read_p1, \
\n" + self.parent1 + "VCF.G_readVCF G_read_p1, " + self.parent1 + "VCF.T_readVCF T_read_p1, " + self.parent1 + "VCF.C_readVCF C_read_p1,\
\n" + self.parent1 + "VCF.A_percentVCF A_percent_p1, " + self.parent1 + "VCF.G_percentVCF G_percent_p1, " + self.parent1 + "VCF.T_percentVCF T_percent_p1, \
\n" + self.parent1 + "VCF.C_percentVCF C_percent_p1 from " + self.proband + self.group + "_genotemp1 left join " + self.parent1 + "VCF on \
\n" + self.proband + self.group + "_genotemp1.varMD5 = " + self.parent1 + "VCF.varMD5VCF;\n"
        query += "DROP TABLE " + self.proband + self.group + "_genotemp1;\n"
        query += "ALTER TABLE " + self.proband + group  + "_genotypes ADD INDEX (uniLoc), ADD INDEX (geneList), ADD INDEX(varMD5);\n"
        self.genotable = self.proband + self.group + "_genotypes"
        return query
    
    def getVariants(self, stringent = False):
        query = ""
        query += "SELECT * FROM " + self.proband + self.group + "_genotypes where \
\n((substring_index(proband_geno, '/', 1) = substring_index(proband_geno, '/', -1)) and \
\n(substring_index(p1_geno, '/', 1) != substring_index(p1_geno, '/', -1))) #loss of heterozygosity \
\nor ((substring_index(proband_geno, '/', 1) != substring_index(proband_geno, '/', -1)) and \
\n(substring_index(p1_geno, '/', 1) = substring_index(p1_geno, '/', -1))) #gain of heterozygosity \
\nor (substring_index(proband_geno, '/', 1) not in \
\n(substring_index(p1_geno, '/', 1), substring_index(p1_geno, '/', -1))) #one allele not found in the parent line\
\nor (substring_index(proband_geno, '/', -1) not in \
\n(substring_index(p1_geno, '/', 1), substring_index(p1_geno, '/', -1))) #the other allele not found in the parent line"
        if not stringent:
            query += ";\n"
        if stringent:
            query += "\nand proband_geno != './.' and p1_geno != './.'; #requires that genotypes were called in both parent and proband;\n;"
        return query
    
    def countVariants(self, stringent = False, unique = False):
        query = ""
        if unique:
            query += "SELECT count(distinct varMD5) FROM "
        if not unique:
            query += "SELECT count(*) FROM "
        query += self.proband + self.group + "_genotypes where \
\n((substring_index(proband_geno, '/', 1) = substring_index(proband_geno, '/', -1)) and \
\n(substring_index(p1_geno, '/', 1) != substring_index(p1_geno, '/', -1))) #loss of heterozygosity \
\nor ((substring_index(proband_geno, '/', 1) != substring_index(proband_geno, '/', -1)) and \
\n(substring_index(p1_geno, '/', 1) = substring_index(p1_geno, '/', -1))) #gain of heterozygosity \
\nor (substring_index(proband_geno, '/', 1) not in \
\n(substring_index(p1_geno, '/', 1), substring_index(p1_geno, '/', -1))) #one allele not found in the parent line\
\nor (substring_index(proband_geno, '/', -1) not in \
\n(substring_index(p1_geno, '/', 1), substring_index(p1_geno, '/', -1))) #the other allele not found in the parent line"
        if not stringent:
            query += ";\n"
        if stringent:
            query += "\nand proband_geno != './.' and p1_geno != './.'; #requires that genotypes were called in both parent and proband;\n;"
        return query
    
    
class CleanUp(object):
    
    def rowstring(data, delimiter = "\t", prepend = "", append = ""):
        rowcollector = ""
        for row in range(0,len(data)):
            if delimiter == "\t":
                if prepend:
                    columncollector = prepend + delimiter
                else:
                    columncollector = ""
            else:
                if prepend:
                    columncollector = '"' + prepend + '"' + delimiter + '"'
                else:
                    columncollector = '"'
            for column in range(0,len(data[row])):
                columncollector += str(data[row][column])
                if not (len(data) == 1 and len(data[row]) == 1):
                    if column == len(data[row]) -1:
                        if delimiter == "\t":
                            if append:
                                columncollector += delimiter + append + delimiter + "\n"
                            else:
                                columncollector += "\n"
                        else:
                            if append:
                                columncollector += '"' + delimiter + '"' + append + '"' + "\n"
                            else:
                                columncollector += '"' + "\n"
                    else:
                        if delimiter == "\t":
                            columncollector += delimiter
                        else:
                            columncollector += '"' + delimiter + '"'
            rowcollector += columncollector
        return rowcollector
    
    def columnstring(data, delimiter = "\t"):
        if delimiter == "\t":
            rowcollector = ""
        else:
            rowcollector = '"'
        for row in range(0,len(data)):
            rowcollector += "".join(data[row])
            if row == len(data) -1:
                if delimiter == "\t":
                    rowcollector += "\n"
                else:
                    rowcollector += '"' + "\n"
            else:
                if delimiter == "\t":
                    rowcollector += delimiter
                else:
                    rowcollector += '"' + delimiter + '"'
        return rowcollector
    
    def columnarray(data):
        datastring = columnstring(data)
        return datastring.split("\t")
    
    def rowarray(data):
        datastring = rowstring(data)
        datastring = datastring.split("\n")
        if len(datastring) <=1:
            dataarray = []
            dataarray[0] = datastring.split("\t")
            return dataarray
        else:
            datastring = [line.split("\t") for line in datastring]
            return datastring
        
        
class Sample(object):
    
    def __init__(self, samples):
        try:
            self.samples = int(samples)
        except:
            raise ValueError("Number of samples to collect must be an integer (or string of an integer, if you really must)")
        self.collected = 0
        self.collection = []
        
    def collect(self, data):
        if self.collected < self.samples:
            self.collection.append(data)
            self.collected += 1
            
    def display(self):
        for item in self.collection:
            print(item)
            
            
class JoinWithVCFTable(object):
    
    def __init__(self, table, suffix = "_all"):
        self.table = table
        self.suffix = suffix
        
    def buildTables(self):
        query = ""
        query += "CREATE TABLE " + self.table + self.suffix + " as \
\nSELECT * from " + self.table + " left join " + self.table + "VCF on varMD5 = varMD5VCF;"
        self.joinedTable = self.table + self.suffix
        return query
    
def getColumns(schema, table):
    query = "SELECT `COLUMN_NAME` FROM `INFORMATION_SCHEMA`.`COLUMNS` WHERE `TABLE_SCHEMA`='" + schema + "' AND `TABLE_NAME`='" + table + "';"
    return query
    
def queryBuilderHelp():
    print("RareCodingVariants(tablename, suffix) Finds variants annotated as less than 1% allele frequency in all tested populations that are annotated as variant types with the potential to affect protein sequence. Suffix for new table defaults to _rare_coding if not set\n\tbuildTables()\n\tgetVariants()\n\tcountVariants()")
    print("KnownGenes(tablename, knownGeneList) Finds variants from the given table in the list of known genes.  knownGeneList should be a list or tupple.\n\tlistFromFile(fileObject) (builds a list of genes from a text file, ignores lines starting with a # symbol)\n\thashedListFromFile(fileObject) (builds multiple lists of files under one dictionary using lines starting with a # symbol as the key)\n\tgetVariants()")
    print("Homozygous(tablename, readCutoff, suffix) Creates a table of homozygous variants from the tablename entered.\nRead cutoff allows for including variants not called as homozygous by the variant caller.\nNew table suffix defaults to _homozygosity.\n\tbuildTables()\n\tgetVariants()\n\tcountVariants(unique) If the same variant is listed multiple times due to listing each transcript, setting this to True will only count it once (default value is False)")
    print("CompoundHet(tablename, suffix) Creates a table of genes with multiple variants.  Default suffix is _compound_het.  VARIANTS MAY OR MAY NOT BE IN PHASE\n\tbuildTables()\n\tgetVariants()\n\tcountVariants(unique)\n\tcountGenes() Returns a count of the genes with multiple variants")
    print("DeNovoChanges(proband, parent1, parent2, group) Finds variants that have a variant genotype in the proband not found in one of the parents.  Group defaults to an empty string, but can be used to add a suffix to the proband's tablename.\n\tbuildTables()\n\tgetVariants(stringent) Stringent is a boolean value (True or False) with a default value of False.  If true, will leave out any loci that were not called in one or both parents.\n\tcountVariants(stringent, unique)")
    print("SomaticChanges(proband, parent1, group) Similar to above, but looks between proband and one parent for any genotypes in the proband not found in the parent, gains of heterozygosity, or losses of heterozygosity.\n\tbuildTables()\n\tgetVariants(stringent)\n\tcountVariants(stringent, unique)")
    print("CleanUp() A holder for functions that turn the SQL output into clean strings or lists\n\trowstring(data, delimiter) Returns a delimited string (default is tab) of the submitted data.\n\tcolumnstring(data, delimiter) Turns a 2-dimensional output into a 1-line dimensional string (useful for column headers)\n\trowarray(data) Returns a 2-dimensional array of data\n\tcolumnarray(data)")
    print("Sample(numberOfSamples) Sets up an object to store the number of samples given.\n\tcollect(data) Stores the data as a sample.\n\tdisplay() Prints the collected samples.")
    print("JoinWithVCFTable(tablename, suffix) Joins the annotation (SeattleSeq) data with the matching VCF data for that variant in the tablename submitted.  Suffix default is _all.\n\tbuildTables()")
    print("getColumns(schema, tablename) Gets a list of column names from the schema and tablename.  Data returned as a column of tupples each containing one column name. (Use columnstring or column array to clean).")   

In [ ]:
#importing the library for communications with the SQL server, creating an objects to handle the connection/commands
schema = "[schemaname]"
import pymysql as mysql
db = mysql.connect(host="127.0.0.1",user="root",passwd="",db=schema)
dbconnection = db.cursor()